In [4]:
import sys
import numpy as np
import scipy.stats
import operator
import itertools
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from scipy.stats import ttest_ind

import multiprocessing

"""
import random

random.seed(int(sys.argv[1]))
np.random.seed(int(sys.argv[1]))
"""

np.seterr(divide='ignore')

#def runGA():
# Generate the distributions to be used in the GP optimization process

# Index by: [sample number (0-29)]
sig_diff_samples_0v10 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_10v0 = np.random.normal(loc=10, scale=5, size=(30, 500))

sig_diff_samples_0v10_same_std = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_10v0_same_std = np.random.normal(loc=100, scale=3, size=(30, 500))

sig_diff_samples_0v100 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_100v0 = np.random.normal(loc=100, scale=5, size=(30, 500))

sig_diff_samples_0v100_same_std = np.random.normal(loc=0, scale=30, size=(30, 500))
sig_diff_samples_100v0_same_std = np.random.normal(loc=100, scale=30, size=(30, 500))

sig_diff_samples_0v1000 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_1000v0 = np.random.normal(loc=1000, scale=5, size=(30, 500))

sig_diff_samples_0v1000_same_std = np.random.normal(loc=0, scale=300, size=(30, 500))
sig_diff_samples_1000v0_same_std = np.random.normal(loc=1000, scale=300, size=(30, 500))

# Index by: [group (0/1)][sample number (0-29)]
both_dist = np.append(sig_diff_samples_0v10.reshape(
                          sig_diff_samples_0v10.shape[0] * sig_diff_samples_0v10.shape[1]),
                      sig_diff_samples_10v0.reshape(
                          sig_diff_samples_10v0.shape[0] * sig_diff_samples_10v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v10 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v10 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v10_same_std.reshape(
                          sig_diff_samples_0v10_same_std.shape[0] * sig_diff_samples_0v10_same_std.shape[1]),
                      sig_diff_samples_10v0_same_std.reshape(
                          sig_diff_samples_10v0_same_std.shape[0] * sig_diff_samples_10v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v10_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v10_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v100.reshape(
                          sig_diff_samples_0v100.shape[0] * sig_diff_samples_0v100.shape[1]),
                      sig_diff_samples_100v0.reshape(
                          sig_diff_samples_100v0.shape[0] * sig_diff_samples_100v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v100 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v100 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v100_same_std.reshape(
                          sig_diff_samples_0v100_same_std.shape[0] * sig_diff_samples_0v100_same_std.shape[1]),
                      sig_diff_samples_100v0_same_std.reshape(
                          sig_diff_samples_100v0_same_std.shape[0] * sig_diff_samples_100v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v100_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v100_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v1000.reshape(
                          sig_diff_samples_0v1000.shape[0] * sig_diff_samples_0v1000.shape[1]),
                      sig_diff_samples_1000v0.reshape(
                          sig_diff_samples_1000v0.shape[0] * sig_diff_samples_1000v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v1000 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v1000 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v1000_same_std.reshape(
                          sig_diff_samples_0v1000_same_std.shape[0] * sig_diff_samples_0v1000_same_std.shape[1]),
                      sig_diff_samples_1000v0_same_std.reshape(
                          sig_diff_samples_1000v0_same_std.shape[0] * sig_diff_samples_1000v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v1000_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v1000_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

ttest_vals = []

# Sig diff sample comparisons
for sample1, sample2 in zip(sig_diff_samples_0v10, sig_diff_samples_10v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v100, sig_diff_samples_100v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v1000, sig_diff_samples_1000v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v10_same_std, sig_diff_samples_10v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v100_same_std, sig_diff_samples_100v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v1000_same_std, sig_diff_samples_1000v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

# Null sample comparisons
for sample1, sample2 in zip(null_samples_0v10[0], null_samples_0v10[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v100[0], null_samples_0v100[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v1000[0], null_samples_0v1000[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v10_same_std[0], null_samples_0v10_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v100_same_std[0], null_samples_0v100_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v1000_same_std[0], null_samples_0v1000_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

# Permuted sample comparisons
for sample1, sample2 in zip(permuted_samples_0v10[0], permuted_samples_0v10[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v100[0], permuted_samples_0v100[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v1000[0], permuted_samples_0v1000[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v10_same_std[0], permuted_samples_0v10_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v100_same_std[0], permuted_samples_0v100_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v1000_same_std[0], permuted_samples_0v1000_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

ttest_vals = np.array(ttest_vals)

# GP tree: takes two arrays as input, returns a test staistic
pset = gp.PrimitiveSetTyped('MAIN', [np.ndarray, np.ndarray], float)
pset.renameArguments(ARG0='x1')
pset.renameArguments(ARG1='x2')

# Logical operators on the distance array
#pset.addPrimitive(np.logical_and, [np.ndarray, np.ndarray], np.ndarray, name='array_and')
#pset.addPrimitive(np.logical_or, [np.ndarray, np.ndarray], np.ndarray, name='array_or')
#pset.addPrimitive(np.logical_xor, [np.ndarray, np.ndarray], np.ndarray, name='array_xor')
#pset.addPrimitive(np.logical_not, [np.ndarray], np.ndarray, name='array_not')

# Mathematical operators on the distance array
#pset.addPrimitive(np.add, [np.ndarray, np.ndarray], np.ndarray, name='array_add')
#pset.addPrimitive(np.subtract, [np.ndarray, np.ndarray], np.ndarray, name='array_sub')
#pset.addPrimitive(np.multiply, [np.ndarray, np.ndarray], np.ndarray, name='array_mul')
#pset.addPrimitive(np.divide, [np.ndarray, np.ndarray], np.ndarray, name='array_div')
#pset.addPrimitive(np.sqrt, [np.ndarray], np.ndarray, name='array_sqrt')
#pset.addPrimitive(np.square, [np.ndarray], np.ndarray, name='array_square')
#pset.addPrimitive(np.abs, [np.ndarray], np.ndarray, name='array_abs')

# Statistics derived from the distance array
pset.addPrimitive(np.mean, [np.ndarray], float, name='array_mean')
#pset.addPrimitive(np.median, [np.ndarray], float, name='array_median')
#pset.addPrimitive(np.min, [np.ndarray], float, name='array_min')
#pset.addPrimitive(np.max, [np.ndarray], float, name='array_max')
#pset.addPrimitive(np.std, [np.ndarray], float, name='array_std')
pset.addPrimitive(np.var, [np.ndarray], float, name='array_var')
pset.addPrimitive(np.size, [np.ndarray], float, name='array_size')
#pset.addPrimitive(np.sum, [np.ndarray], float, name='array_sum')
pset.addPrimitive(scipy.stats.sem, [np.ndarray], float, name='array_stderr')

# Mathematical operators with single values
def protected_div(left, right):
    try:
        return float(left) / float(right)
    except ZeroDivisionError:
        return 1.

pset.addPrimitive(operator.add, [float, float], float, name='float_add')
pset.addPrimitive(operator.sub, [float, float], float, name='float_sub')
#pset.addPrimitive(operator.mul, [float, float], float, name='float_mul')
pset.addPrimitive(protected_div, [float, float], float, name='float_div')
pset.addPrimitive(np.sqrt, [float], float, name='float_sqrt')
#pset.addPrimitive(np.square, [float], float, name='float_square')
#pset.addPrimitive(np.abs, [float], float, name='float_abs')

# Mathematical operators on the distance array with a single value
pset.addPrimitive(np.add, [np.ndarray, float], np.ndarray, name='array_add_float')
#pset.addPrimitive(np.subtract, [np.ndarray, float], np.ndarray, name='array_sub_float')
#pset.addPrimitive(np.multiply, [np.ndarray, float], np.ndarray, name='array_mul_float')
#pset.addPrimitive(np.divide, [np.ndarray, float], np.ndarray, name='array_div_float')

# Equivalence operators on the distance array with a single value
#pset.addPrimitive(np.less, [np.ndarray, float], np.ndarray, name='array_less_than_float')
#pset.addPrimitive(np.equal, [np.ndarray, float], np.ndarray, name='array_equal_float')

# Terminals
pset.addTerminal(1.0, float)
#pset.addEphemeralConstant('rand{}'.format(np.random.randint(1e9)), lambda: np.random.random() * 100., float)
#pset.addTerminal(np.multiply(np.random.random(size=features.shape[0]), 100.), np.ndarray)
#pset.addTerminal(np.array([True] * features.shape[0]), np.ndarray)
#pset.addTerminal(np.array([False] * features.shape[0]), np.ndarray)

creator.create('FitnessMulti', base.Fitness, weights=(-1., -1.))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=4)

"""
def return_ttest():
    return creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_add(array_stderr(x1), array_stderr(x2)))', pset)
    #return creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_sqrt(float_add(float_div(array_var(x1), array_size(x1)), float_div(array_var(x2), array_size(x2)))))', pset)

toolbox.register('individual', return_ttest)
"""
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

pool = multiprocessing.Pool()
toolbox.register('map', pool.map)

def evaluate_individual(individual):
    # If the solution contains a constant value in it, throw it out
    str_ind = str(individual)
    if '1.0' in str_ind:
        return sys.maxsize, sys.maxsize
    
    func = toolbox.compile(expr=individual)

    gp_ttest_vals = []
    
    # Sig diff sample comparisons
    for sample1, sample2 in zip(sig_diff_samples_0v10, sig_diff_samples_10v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v100, sig_diff_samples_100v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v1000, sig_diff_samples_1000v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))
    
    for sample1, sample2 in zip(sig_diff_samples_0v10_same_std, sig_diff_samples_10v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v100_same_std, sig_diff_samples_100v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v1000_same_std, sig_diff_samples_1000v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    # Null sample comparisons
    for sample1, sample2 in zip(null_samples_0v10[0], null_samples_0v10[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v100[0], null_samples_0v100[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v1000[0], null_samples_0v1000[1]):
        gp_ttest_vals.append(func(sample1, sample2))

    for sample1, sample2 in zip(null_samples_0v10_same_std[0], null_samples_0v10_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))

    for sample1, sample2 in zip(null_samples_0v100_same_std[0], null_samples_0v100_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v1000_same_std[0], null_samples_0v1000_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    # Permuted sample comparisons
    for sample1, sample2 in zip(permuted_samples_0v10[0], permuted_samples_0v10[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v100[0], permuted_samples_0v100[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v1000[0], permuted_samples_0v1000[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    for sample1, sample2 in zip(permuted_samples_0v10_same_std[0], permuted_samples_0v10_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    for sample1, sample2 in zip(permuted_samples_0v100_same_std[0], permuted_samples_0v100_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v1000_same_std[0], permuted_samples_0v1000_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    gp_ttest_vals = np.array(gp_ttest_vals)
        
    # If the solution produces NaN or inf values, throw it out
    if np.any(np.isnan(gp_ttest_vals)) or np.any(np.isinf(gp_ttest_vals)):
        return sys.maxsize, len(individual)

    # Third fitness component is the size (i.e., complexity) of the GP tree
    ind_complexity = np.sum([type(component) == gp.Primitive for component in individual])
    
    return (
        np.sum(np.abs(np.subtract(gp_ttest_vals, ttest_vals))),
        ind_complexity
           )

toolbox.register('evaluate', evaluate_individual)
toolbox.register('select', tools.selNSGA2)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr_mut', gp.genFull, min_=0, max_=3)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def pareto_eq(ind1, ind2):
    """Determines whether two individuals are equal on the Pareto front
    Parameters
    ----------
    ind1: DEAP individual from the GP population
        First individual to compare
    ind2: DEAP individual from the GP population
        Second individual to compare
    Returns
    ----------
    individuals_equal: bool
        Boolean indicating whether the two individuals are equal on
        the Pareto front
    """
    return np.allclose(ind1.fitness.values, ind2.fitness.values)

pop = toolbox.population(n=500)
pareto_front = tools.ParetoFront(similar=pareto_eq)
stats = tools.Statistics(lambda ind: ind.fitness.values[0])
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

t_test = creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_sqrt(float_add(float_div(array_var(x1), array_size(x1)), float_div(array_var(x2), array_size(x2)))))', pset)
print('t-test fitness: {}'.format(evaluate_individual(t_test)))

t_test = creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_add(array_stderr(x1), array_stderr(x2)))', pset)
print('t-test fitness: {}'.format(evaluate_individual(t_test)))

algorithms.eaMuPlusLambda(population=pop, toolbox=toolbox,
                          cxpb=0.5, mutpb=0.5, mu=500, lambda_=1000,
                          ngen=250, stats=stats, halloffame=pareto_front)

print('')
for index, ind in enumerate(pareto_front):
    print(index, ind.fitness, ind)

<string>:1: RuntimeWarning: invalid value encountered in sqrt
<string>:1: RuntimeWarning: invalid value encountered in sqrt


t-test fitness: (293.2594521893796, 12)
t-test fitness: (80336.660849789376, 7)


<string>:1: RuntimeWarning: invalid value encountered in sqrt
<string>:1: RuntimeWarning: invalid value encountered in sqrt


gen	nevals	avg        	std        	min   	max        
0  	500   	7.36025e+18	3.70311e+18	293432	9.22337e+18


<string>:1: RuntimeWarning: invalid value encountered in sqrt
<string>:1: RuntimeWarning: invalid value encountered in sqrt
<string>:1: RuntimeWarning: invalid value encountered in sqrt


1  	1000  	4.68547e+18	4.6111e+18 	292721	9.22337e+18


<string>:1: RuntimeWarning: invalid value encountered in sqrt


2  	1000  	1.15395e+07	1.83107e+07	292721	4.16586e+07
3  	1000  	337155     	65660.4    	277472	560818     
4  	1000  	295625     	1334.84    	277472	295902     
5  	1000  	295722     	1658.31    	277472	295888     
6  	1000  	295791     	1212.82    	277472	295888     
7  	1000  	295840     	875.839    	276584	295888     
8  	1000  	295801     	1227.67    	276584	295888     
9  	1000  	295801     	1227.67    	276584	295888     
10 	1000  	295763     	1498.09    	276584	295888     
11 	1000  	295801     	1227.67    	276584	295888     
12 	1000  	295763     	1498.09    	276584	295888     
13 	1000  	295653     	5040.47    	183111	295888     
14 	1000  	295427     	7119.47    	183111	295888     
15 	1000  	295427     	7119.47    	183111	295888     
16 	1000  	295427     	7119.47    	183111	295888     
17 	1000  	295427     	7119.47    	183111	295888     
18 	1000  	295427     	7119.47    	183111	295888     
19 	1000  	294976     	10047      	183111	295888     
20 	1000  	295427     	7119.